In [1]:
import spacy
import spacy_transformers
from nltk.tokenize import sent_tokenize
import os
import re
import xlsxwriter
import pandas

In [2]:
nlp = spacy.load("en_core_web_trf")

In [3]:
def open_file(filename):
    with open(filename, encoding='utf-8') as f:
        text = f.read()
    return text

In [4]:
def writeln(errors, filename):
    workbook = xlsxwriter.Workbook(filename)
    worksheet = workbook.add_worksheet()
    #Fill in the column names
    for x in range(len(errors[0])):
        worksheet.write(0, x, errors[0][x])

    col = 0
    for row_num in range(1, len(errors)):
        error = errors[row_num]
        for x in range(len(error)):
            worksheet.write(row_num, col + x, error[x])
    workbook.close()

In [5]:
def find_subj(pred):
    # print(pred, [child.text for child in pred.lefts]) use pred.lefts to find quotes
    # simple cases
    subjects = []
    subjects = [child for child in list(pred.children) if child.dep_.startswith(('nsubj', 'csubj'))]

    # handling 'there is' and 'there are' cases
    if pred.lemma_ == 'be' and 'there' in list(i.text.lower() for i in pred.children):
        subjects += [child for child in list(pred.children) if child.dep_ == 'attr']
    
    # if predicate is an auxiliary, we want to take subjects of its head
    if pred.dep_.startswith('aux'):
        subjects += [child for child in list(pred.head.children) if child.dep_.startswith(('nsubj', 'csubj'))]
    
    # handling conjuncts: multiple subjects as in 'Mother and father are key figures in a child's life'.
    add_subj = []
    for subject in subjects:
        add_subj += list(subject.conjuncts)

    cur_pred = pred
    while len(subjects) == 0 and cur_pred.dep_ == "conj":
        cur_pred = cur_pred.head
        subjects = find_subj(cur_pred)
            
    subjects += add_subj

    # the subjects' order may be different from sentence order, so we arrange it right
    subjects.sort(key=lambda subj: subj.i)
        
    return subjects

In [6]:
def find_pred_subj(doc):
    pred_sub = list()
    for token in doc:
        if token.pos_ in ['AUX', 'VERB']:
            # negation: cases like "He doesn't scare me"   
            aux = None
            if [ch for ch in list(token.lefts) if ch.dep_ == 'neg']:
                children = list(token.children)
                for ch in children:
                    if ch.dep_ == 'aux' and not aux:
                        aux = ch
                if aux:
                    pred_sub += [(aux, find_subj(aux))]
                    
                    
            # if the predicate is analytical like 'I have done',
            # spacy rightfully considers the participle to be the root,
            # but we need grammatical info, so we will consider aux the root
            if not aux: # for when negation is expressed with 'never' etc and does not need aux support
                if token.tag_ in ['VBN', 'VBG']:
                    aux = None
                    children = list(token.children)
                    for ch in children:
                        if ch.dep_ == 'aux' and ch.pos_ in ['VERB', 'AUX'] and ch.tag_ != 'VBN':
                            aux = ch
                        elif not aux and ch.dep_ == 'auxpass' and ch.pos_ in ['VERB', 'AUX']:
                            aux = ch
                    if aux:
                        pred_sub += [(aux, find_subj(aux))]

                # all other cases
                elif token.dep_ in ['ROOT', 'ccomp', 'xcomp', 'acl', 'relcl']:
                    pred_sub += [(token, find_subj(token))]

                # conjuncts: when there are multiple predicates connected by conjunction
                elif token.dep_ == 'conj' and token.head.dep_ in ['ROOT', 'ccomp', 'xcomp', 'acl', 'relcl']:
                    pred_sub += [(token, find_subj(token))]
                
    return pred_sub

In [21]:
def errors(ps):
    res = []
    for pair in ps:
        subj_agr, pred_agr = None, None #what each must agree with, variables must coincide in the end
        pred = pair[0]
        subj = pair[1]
        print(pred, subj)
        if len(subj) == 1:
            subject = subj[0]
            s = subject.text.lower()
            subject_left_children = subject.lefts
            subject_is_numeral = False
            for child in subject_left_children:
                if child.pos_ == 'NUM':
                    subject_is_numeral = True
                    break
            if subject_is_numeral:
                continue
            if s not in ambiguous:
                children = list(ch for ch in subject.children)
                children_text = list(ch.text.lower() for ch in children)
                
                #singular only pronouns
                if s in sing_only or subject.ent_type_ == 'ORG':
                    subj_agr = 'sg'
                elif subject.tag_ == 'VB':
                    subj_agr = 'sg'
                    
                # either singular or plural pronouns
                # if they have an 'of N, N, N...', after them we will require check if verb agrees with the last noun
                elif (s in {'some', 'any', 'none', 'all', 'most'} \
                      and 'of' in children_text):
                    of = [ch for ch in children if ch.text.lower() == 'of'][0]
                    noun = [ch for ch in of.children if ch.pos_ == 'NOUN']
                    if noun:
                        noun = noun[0]
                        while [ch for ch in noun.children if (ch.dep_ == 'conj' and ch.pos_ == 'NOUN')]:
                            noun = [ch for ch in noun.children if (ch.dep_ == 'conj' and ch.pos_ == 'NOUN')][-1]
                        if noun.tag_ in ['NNS', 'NNPS'] or noun.text.lower() in plur_only:
                            subj_agr = 'pl'
                        elif noun.tag_ in ['NN', 'NNP'] or noun.text.lower() in sing_only:
                            subj_agr = 'sg'
                    
                # plural only pronouns
                elif s in plur_only and not children:
                    subj_agr = 'pl'
                    
                elif s in {'i', 'we', 'you', 'they'}:
                    subj_agr = 'pl'
                elif s in {'he', 'she', 'it'}:
                    subj_agr = 'sg'
                    
                elif s == 'number':
                    if 'a' in children_text and 'of' in children_text:
                        subj_agr = 'pl'
                    else: subj_agr = 'sg'
                    
                # predicates in non-head clauses with 'who', 'that' agree with noun in head clause
                elif s in ['who', 'that', 'which']:
                    if pred.dep_ == 'relcl':
                        #why relcl? we can only be sure about this tag that it is the case we're looking for.
                        #other possible predicate tags include '...comp', but these also apply
                        #in cases like "I asked the boys who was the winner",
                        #which, although with incorrect word order,
                        #are still clearly present in Russian essays
                        #and will be parsed by spacy as 'ccomp'
                        head = pred.head
                        if not head.conjuncts:
                            if head.tag_ in ['NNS', 'NNPS'] or head.text.lower() in plur_only:
                                subj_agr = 'pl'
                            elif head.tag_ in ['NN', 'NNP'] or head.text.lower() in sing_only:
                                subj_agr = 'sg'
                        else:
                            conjuncts = list(head.conjuncts)+[head]
                            for conjunct in conjuncts:
                                if 'and' in list(child.text.lower() for child in conjunct.children):
                                    subj_agr = 'pl'     
                elif subject.tag_ in ['NNS', 'NNPS']:
                    subj_agr = 'pl'
                elif subject.tag_ in ['NN', 'NNP', 'VBG']:
                    subj_agr = 'sg'
                if subject.ent_type_ == 'LOC' or subject.ent_type_ == 'GPE':
                    if subject.tag_ in ['NNS', 'NNPS']:
                        continue
                    else:
                        subj_arg = 'sg'
                    
        elif len(subj) > 1:
            # 'Mother, father and brother were present.'
            # If conjuncts are connected by 'and', he predicate is plural
            # Exception: 'Every man, woman and child aprticipates in the tournament.'
            if 'and' in list(child.text.lower() for child in list(list(subj[0].children)+list(subj[-2].children))):
                left_subj_children = list(child.text.lower() for child in list(subj[0].lefts))
                left_pred_children = list(child.text.lower() for child in list(pred.lefts))
                is_uppercase = True
                all_gerund = True
                for sub in subj:
                    if sub.tag_ != 'NNP':
                        is_uppercase = False
                        break
                    if sub.tag_ != 'VBG':
                        all_gerund = False
                #Don't check: 'Playing football and enjoying it + is a good thing | are different things)'
                if all_gerund:
                    continue
                #Don't check: 'Jones and Sons is a respectable company.'
                if is_uppercase:
                    continue
                # Don't check: 'There is Tom and Mary as a perfect example.'
                if pred.text.lower() in ['is', 'are'] and 'there' in left_pred_children:
                    continue
                if 'every' in left_subj_children or 'each' in left_subj_children:
                    subj_agr = 'sg'
                else:
                    subj_agr = 'pl'
            # 'Mother, father or brother comes to pick up the kid.'
            # If conjuncts are connected by 'or', verb agrees with the last one
            elif any(a in list(child.text.lower() for child in list(list(subj[0].children)+list(subj[-2].children))) for a in ['or', 'nor']):
                if subj[-1].tag_ in ['NNS', 'NNPS'] or subj[-1].text.lower() in plur_only:
                    subj_agr = 'pl'
                elif subj[-1].tag_ in ['NN', 'NNP', 'VBG'] or subj[-1].text.lower() in sing_only:
                    subj_agr = 'sg'

        if pred.tag_ == 'VBZ':
            pred_agr = 'sg'
        elif pred.tag_ == 'VBP':
            pred_agr = 'pl'
        elif pred.lemma == 'be':
            if pred.text == 'was':
                pred_agr = 'sg'
            elif pred.text == 'were':
                pred_agr = 'pl'

        if subj_agr != pred_agr and subj_agr and pred_agr:
            res += [pair]
            
    return res

In [8]:
def search(directory = 'test', only_errors=True): #If used with excel files, should be only used with similar format files, otherwise additional info can be messy
    final = []
    text_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(directory) for f in filenames if os.path.splitext(f)[1] == '.txt']
    excel_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(directory) for f in filenames if os.path.splitext(f)[1] == '.xlsx']

    if text_files: #If there is at least one txt file, we assume that the format is default
        if only_errors:
            final.append(("Sentence", "Errors", "Filename"))
        else:
            final.append(("Sentence", "Errors", "Filename", "Has Error"))

    for file in text_files:
        text = open_file(file)
        sentences = sent_tokenize(text)
        for sent in sentences:
            sent = re.sub(r'<.*?>', '', sent)
            sent = re.sub(r' +', ' ', sent)
            doc = nlp(sent)
            ps = find_pred_subj(doc)
            # ps looks like [(predicate1, [subject1, subject2]), (predicate2, [subject1])]
            er = errors(ps)
#             er = [error for error in er if not subjunctive(sent, error[0])]
            
            if only_errors:
                if er:
                    final.append((sent,
                                  ",\n".join(["{} {}".format(*er[x]) for x in range(len(er))]), # Wrong subjs and preds,
                                 file))
            else:
                final.append((sent,
                              ", ".join(["({} {})".format(*er[x]) for x in range(len(er))]),
                              file,
                              int(len(er) != 0)))  #Whether or not errors were found in this sentence

    for file in excel_files:
        df = pandas.read_excel(file)
        if not final:
            additional_columns = tuple([x for x in df.columns if x != "sentence"])
            if only_errors:
                final.append(("Sentence", "Errors", "Filename") + additional_columns)
            else:
                final.append(("Sentence", "Errors", "Filename", "Has Error") + additional_columns)

        for row_num in range(df.shape[0]):
            sent = df["sentence"].iat[row_num]
            sent = re.sub(r'<.*?>', '', sent)
            sent = re.sub(r' +', ' ', sent)
            doc = nlp(sent)
            ps = find_pred_subj(doc)
            # ps looks like [(predicate1, [subject1, subject2]), (predicate2, [subject1])]
            er = errors(ps)
#             er = [error for error in er if not subjunctive(sent, error[0])]
            
            if only_errors:
                if er:
                    final.append((sent,
                                  ",\n".join(["{} {}".format(*er[x]) for x in range(len(er))]), # Wrong subjs and preds,
                                 file,
                                 *[df[column].iat[row_num] for column in additional_columns]))
            else:
                final.append((sent,
                              ", ".join(["({} {})".format(*er[x]) for x in range(len(er))]),
                              file,
                              int(len(er) != 0), #Whether or not errors were found
                              *[df[column].iat[row_num] for column in additional_columns]))
        
    return final

In [9]:
def subjunctive(sent, pred):
    adj_prt = {'advisable', 'best', 'crucial', 'desirable', 'vital',
               'essential', 'imperative', 'important', 'moved',
               'necessary', 'prohibited', 'unthinkable', 'urgent',
               'required', 'stipulated', 'requested', 'recommended',
               'advised', 'proposed', 'adamant', 'anxious', 
               'determined', 'eager', 'insistent', 'keen'}
    
    clause_is_subjunctive = False
    that_list = []

    if 'that' in sent.lower():
        doc = nlp(sent)
        that_list = [token for token in doc if token.text.lower() == 'that']
        for that in that_list: 
            # VERB/NOUN + that + CONJUNCTIVE
            if (that.head.i == pred.i \
                and pred.dep_ == 'ccomp' \
                and pred.head.pos_ in ['VERB', 'NOUN'] \
                and pred.head.lemma_ in {'advise', 'ask', 'command', 'demand', 'desire',
                                         'insist', 'move', 'order', 'prefer', 'propose',
                                         'recommend', 'request', 'stipulate', 'suggest', 'urge',
                                         'motion', 'order', 'preference', 'proposal', 'recommendation',
                                         'request', 'stipulation', 'suggestion'}):
                clause_is_subjunctive = True

            # SUBJECT + [be] ADJ/PRTC + that + CONJUNCTIVE
            elif (that.head.i == pred.i and pred.dep_ == 'ccomp'):
                if (pred.head.lemma_ == 'be' and \
                    any(a in adj_prt for a in list(b.text.lower() for b in pred.head.children))) \
                    or (pred.head.tag_ in ['VBN', 'JJ'] and \
                        pred.head.text in adj_prt):
                        clause_is_subjunctive = True

    return clause_is_subjunctive

In [17]:
ambiguous = {'bison', 'cod', 'deer', 'fish', 'moose', 'boar', 'salmon', 'sheep',
            'shrimp', 'swine', 'trout', 'buffalo', 'grouse', 'elk', 'fruit', 'reindeer',
            'offspring', 'pike',
            'statistics', 'politics', 'mechanics', 'economics',            
            'government', 'data', 'police', 'team', 'jury', 'family',
            'half', 'class', 'majority', 'part', 'percent', '%', 'cent', 'lot'}

sing_only = {'each', 'either', 'neither', 'one', 'nobody',
            'nothing', 'anyone', 'anybody', 'anything', 'someone', 
            'somebody', 'something', 'everyone', 'everybody', 'everything', 
             'this', 'one', 'other'}

plur_only = {'several', 'few', 'many', 'both', 'these', 'those'}

In [229]:
%%time
f = search('../razmetka/', False)

CPU times: user 2min 53s, sys: 152 ms, total: 2min 53s
Wall time: 43.4 s


In [230]:
writeln(f, "itog.xlsx")

## Test

In [11]:
def run_model(*args):
    return search(os.path.join(*args), False)

In [12]:
def test_model(*args, write_errs=None, print_wrong=False): # Function to test the model on a directory with a correct.txt file and a wrong.txt file
    output = run_model(*args)
    
    if write_errs is not None:
        writeln(output, write_errs)
    
    false_pos = 0
    true_pos = 0
    false_neg = 0
    true_neg = 0
    for x in output[1:]:
        if x[2] == os.path.join(*args, "correct.txt"):
            if x[3] == 0:
                true_neg += 1
            else:
                if print_wrong:
                    print("False positive:", x)
                false_pos += 1
        else:
            if x[3] == 0:
                if print_wrong:
                    print("False negative:", x)
                false_neg += 1
            else:
                true_pos += 1
    precision = (true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg)
    recall = true_pos / max(1, true_pos + false_neg)
    print("Precision = {}\nRecall = {}\n".format(round(precision, 2), round(recall, 2)))

def evaluate(level, **kwargs): # level should be equal to "easy", "medium" or "insane"
    print(level.capitalize(), "tests:")
    test_model("test", "evaluate", level, **kwargs)

In [18]:
test_model("test/test_wa", print_wrong=True, write_errs="output.xlsx")

do [logic]
act [concerns]
False positive: ('Capitalist logic of investments and incomes do not work in such case musical label, music-concerns do act in a role of giant parasite.', '(do [logic])', 'test/test_wa/correct.txt', 1)
Precision = 0.0
Recall = 0.0



## Example

In [27]:
s = ["measles is stupid"]
for ss in s:
    doc = nlp(ss)
    print(doc)
    print(find_pred_subj(doc))
    for token in doc:
        print("%s\t%s\t\t\t%s\t%s\t\t%s\t\t%s\t\t%s" % (token.i, token.text, token.pos_, 
                                                        token.lemma_, token.tag_, token.dep_, token.head.i))
    
#print(dir(doc[0]))
#print(' '.join(list(ch.text for ch in doc[3].children)))

measles is stupid
[(is, [measles])]
0	measles			PROPN	measles		NNP		nsubj		1
1	is			AUX	be		VBZ		ROOT		1
2	stupid			ADJ	stupid		JJ		acomp		1


In [ ]:
spacy.explain("pobj")

'object of preposition'

In [130]:
st = sent_tokenize("Apples is good")